# Voting Results by Geography Table Exporter

In [27]:
import os
import sqlalchemy
import pandas as pd
import datetime
import ipywidgets as widgets

In [2]:
# enables pandas copy-on-write globally
pd.set_option("mode.copy_on_write", True)

## User Input
Input county FIPS ID and election data to be analyzed.

In [4]:
county_fips = widgets.BoundedIntText(
    value=453,
    min=1,
    max=999,
    step=1,
    # description='Text:',
    disabled=False
)
county_fips

BoundedIntText(value=453, max=999, min=1)

In [5]:
election_years_available = ['2022']
election_year = widgets.Dropdown(
    options = election_years_available,
    value = '2022',
    disabled=False
)
election_year

Dropdown(options=('2022',), value='2022')

In [6]:
db_filepath = os.path.join('data', 'databases', 'texas_legislative_council', f'texas_legislative_council_election_dataset_{election_year.value}.db')
sql_engine = sqlalchemy.create_engine('sqlite:///' + db_filepath)

In [32]:
df = pd.read_sql('election_returns', sql_engine)

In [33]:
df.head(), df.shape

(   index County  FIPS   VTD cntyvtd  vtdkeyvalue       Office    Name Party  \
 0  10450  Bexar    29  1001  291001          251  U.S. Rep 35   Casar     D   
 1  10451  Bexar    29  1002  291002          252  U.S. Rep 20  Castro     D   
 2  10452  Bexar    29  1003  291003          253  U.S. Rep 35   Casar     D   
 3  10453  Bexar    29  1004  291004          254  U.S. Rep 20  Castro     D   
 4  10454  Bexar    29  1005  291005          255  U.S. Rep 20  Castro     D   
 
   Incumbent  Votes  
 0         N    416  
 1         Y    185  
 2         N    718  
 3         Y    274  
 4         Y    196  ,
 (114403, 11))

In [34]:
df = df[df['FIPS'] == county_fips.value]
df.head(), df.shape

(        index  County  FIPS   VTD  cntyvtd  vtdkeyvalue       Office     Name  \
 82311  635664  Travis   453  0100  4530100         8706  U.S. Rep 35    Casar   
 82312  635665  Travis   453  0101  4530101         8707  U.S. Rep 35    Casar   
 82313  635666  Travis   453  0102  4530102         8708  U.S. Rep 35    Casar   
 82314  635667  Travis   453  0103  4530103         8709  U.S. Rep 35    Casar   
 82315  635668  Travis   453  0104  4530104         8710  U.S. Rep 37  Doggett   
 
       Party Incumbent  Votes  
 82311     D         N   1415  
 82312     D         N   1220  
 82313     D         N   1601  
 82314     D         N   1091  
 82315     D         Y   1976  ,
 (32092, 11))

In [36]:
offices = pd.unique(df['Office'])
offices.tolist()
office = widgets.Dropdown(
    options = offices.tolist(),
    disabled=False
)
office

Dropdown(options=('U.S. Rep 35', 'U.S. Rep 37', 'U.S. Rep 10', 'U.S. Rep 17', 'U.S. Rep 21', 'Governor', 'Lt. …

In [37]:
df = df[df['Office'] == office.value]
df.columns = [x.lower() for x in df.columns]
df.head(),  df.shape

(        index  county  fips   vtd  cntyvtd  vtdkeyvalue       office   name  \
 82311  635664  Travis   453  0100  4530100         8706  U.S. Rep 35  Casar   
 82312  635665  Travis   453  0101  4530101         8707  U.S. Rep 35  Casar   
 82313  635666  Travis   453  0102  4530102         8708  U.S. Rep 35  Casar   
 82314  635667  Travis   453  0103  4530103         8709  U.S. Rep 35  Casar   
 82316  635669  Travis   453  0105  4530105         8711  U.S. Rep 35  Casar   
 
       party incumbent  votes  
 82311     D         N   1415  
 82312     D         N   1220  
 82313     D         N   1601  
 82314     D         N   1091  
 82316     D         N   1428  ,
 (236, 11))

In [40]:
# sets timestamp and data fields to create unique directory name for each set of files created
timestamp = datetime.datetime.strftime(datetime.datetime.now(), "%m-%d-%Y %I:%M:%S%p")
timestamp = timestamp.lower()

output_path_dir = os.path.join('data', 'outputs', 'texas_legislative_council', timestamp)
os.makedirs(output_path_dir, exist_ok=True)

df.to_csv(os.path.join(output_path_dir, f'{county_fips.value} - {office.value.lower()}.csv'))